In [37]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info("test")

INFO:root:test


In [2]:
import os
import pandas as pd
import numpy as np
import nltk
import random
import torch
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import itertools
import random

from nltk.corpus import stopwords
STOP_WORDS = set(stopwords.words('english'))

In [3]:
TRAIN_DATA_DIR = '/run/media/root/Windows/Users/agnes/Downloads/data/msmarco/train_data/splitted'

In [4]:
fn = os.path.join(TRAIN_DATA_DIR, 'queries3_entiredoc_regression', 'queries3_entiredoc_regression_train.csv')

In [5]:
df = pd.read_csv(fn)
df.head(3)

,qid,sent_1,rel,sent_2
0,1,ally auto number,1,Phone Number of Ally Auto is 1-888-925-2559 ....
1,1,ally auto number,1,Phone Number of Ally Auto is 1-888-925-2559 ....
2,1,ally auto number,1,Phone Number of Ally Auto is 1-888-925-2559 ....


In [6]:
raw_docs = list(set(df['sent_1'])) + list(set(df['sent_2']))
random.shuffle(raw_docs)

In [7]:
len(raw_docs)

76154

In [8]:
#raw_docs = raw_docs[:10000]

In [ ]:
def preprocess_text(text):
    words = nltk.word_tokenize(text)
    words = [word.lower() for word in words if word.isalpha()]
    words = [w for w in words if not w in STOP_WORDS]
    return words

In [9]:
processed_docs = []
for i, doc in enumerate(raw_docs):
    processed_docs.append(preprocess_text(doc))
    if (i+1) % 2000 == 0:
        print('processed {:d} of {:d}'.format(i+1, len(raw_docs)))
print('Done.')

processed 2000 of 76154
processed 4000 of 76154
processed 6000 of 76154
processed 8000 of 76154
processed 10000 of 76154
processed 12000 of 76154
processed 14000 of 76154
processed 16000 of 76154
processed 18000 of 76154
processed 20000 of 76154
processed 22000 of 76154
processed 24000 of 76154
processed 26000 of 76154
processed 28000 of 76154
processed 30000 of 76154
processed 32000 of 76154
processed 34000 of 76154
processed 36000 of 76154
processed 38000 of 76154
processed 40000 of 76154
processed 42000 of 76154
processed 44000 of 76154
processed 46000 of 76154
processed 48000 of 76154
processed 50000 of 76154
processed 52000 of 76154
processed 54000 of 76154
processed 56000 of 76154
processed 58000 of 76154
processed 60000 of 76154
processed 62000 of 76154
processed 64000 of 76154
processed 66000 of 76154
processed 68000 of 76154
processed 70000 of 76154
processed 72000 of 76154
processed 74000 of 76154
processed 76000 of 76154
Done.


In [10]:
tagged_documents = [TaggedDocument(pdoc, [i]) for i, pdoc in enumerate(processed_docs)]

In [11]:
def calc_loss(v1, v2, label):
    output = torch.cosine_similarity(torch.Tensor([v1]), torch.Tensor([v2]))
    loss_fct = torch.nn.MSELoss()
    loss = loss_fct(output, torch.Tensor([label]))
    return loss.item()

In [12]:
def get_dev_eval_data(dev_df, model):
    
    dev_df = dev_df.copy()
    
    dev_df['sent_1_pp'] = dev_df['sent_1'].apply(preprocess_text)
    dev_df['sent_2_pp'] = dev_df['sent_2'].apply(preprocess_text)

    all_dev_queries = set(dev_df['qid'])

    dev_df['doc_id'] = 0

    qid_to_vec = {}
    docid_to_vec = {}

    for counter, qid in enumerate(all_dev_queries):
        qid_df = dev_df.loc[dev_df['qid'] == qid].reset_index(drop=True)

        doc_ids = [str(qid) + '_' + str(i) for i in range(len(qid_df))]
        dev_df.loc[dev_df['qid'] == qid, 'doc_id'] = doc_ids

        query = qid_df['sent_1_pp'].iloc[0]
        docs = qid_df['sent_2_pp'].values.tolist()

        qid_to_vec[str(qid)] = model.infer_vector(query)
        for i, doc in enumerate(docs):
            docid_to_vec[doc_ids[i]] = model.infer_vector(doc)

        if (counter+1) % 1000 == 0:
            print('processed {:d} of {:d}'.format(counter+1, len(all_dev_queries)))
            
    return qid_to_vec, docid_to_vec, dev_df

In [13]:
def calc_dev_performance(qid_to_vec, docid_to_vec, eval_dev_df, loss_column_name='loss'):
    eval_dev_df = eval_dev_df.copy()
    losses = []
    for i, row in eval_dev_df.iterrows():
        qid = row['qid']
        docid = row['doc_id']
        label = int(row['rel'])
        loss = calc_loss(qid_to_vec[str(qid)], docid_to_vec[docid], label)
        losses.append(loss)
        
        if (i+1) % 1000 == 0:
            print('calc dev performance: processed {:d} of {:d} rows'.format(i+1, len(eval_dev_df)))
    eval_dev_df[loss_column_name] = losses
    return eval_dev_df


In [14]:
dev_fn = os.path.join(TRAIN_DATA_DIR, 'queries3_entiredoc_regression', 
                      'queries3_entiredoc_regression_dev.csv')
dev_df = pd.read_csv(dev_fn)
dev_df.head(5)

,qid,sent_1,rel,sent_2
0,16,amniocentesis is performed what trimester,1,Amniocentesis in late pregnancy. If you are at...
1,16,amniocentesis is performed what trimester,1,Amniocentesis in late pregnancy. If you are at...
2,16,amniocentesis is performed what trimester,1,Amniocentesis in late pregnancy. If you are at...
3,16,amniocentesis is performed what trimester,1,Amniocentesis in late pregnancy. If you are at...
4,16,amniocentesis is performed what trimester,1,Amniocentesis in late pregnancy. If you are at...


In [22]:
def train_doc2vec(tagged_documents, output_fn, vector_size=1024, window_size=2, dm=1, min_alpha=0.0001,
                  epochs=5):
    
    model = Doc2Vec(tagged_documents, vector_size=vector_size, window=window_size, min_count=1, workers=3,
                    dm=dm, min_alpha=min_alpha, epochs=epochs)
    
    if output_fn is not None:
        model.save(output_fn)
    return model

In [29]:
my_epochs = [3, 5, 100]
my_vector_sizes = [4, 40, 104, 1024, 2048]
my_window_sizes = [2,3]
my_dms = [0,1]
my_min_alphas = [.0001, .00001]

combis = list(itertools.product(my_epochs, my_vector_sizes, my_window_sizes, my_dms, my_min_alphas))

sample_combis = random.sample(combis, min(20, len(combis)))
sample_combis

[(3, 40, 3, 1, 0.0001),
 (5, 2048, 2, 1, 0.0001),
 (100, 1024, 3, 0, 0.0001),
 (5, 104, 3, 1, 1e-05),
 (100, 2048, 3, 0, 0.0001),
 (3, 2048, 2, 0, 0.0001),
 (5, 4, 2, 1, 0.0001),
 (5, 2048, 2, 0, 0.0001),
 (100, 4, 2, 0, 1e-05),
 (3, 4, 2, 1, 1e-05),
 (3, 1024, 2, 0, 0.0001),
 (5, 1024, 3, 0, 0.0001),
 (5, 104, 3, 1, 0.0001),
 (5, 1024, 3, 0, 1e-05),
 (100, 2048, 2, 1, 1e-05),
 (3, 1024, 3, 0, 1e-05),
 (3, 4, 3, 0, 1e-05),
 (3, 40, 2, 0, 1e-05),
 (3, 104, 3, 0, 1e-05),
 (100, 4, 3, 1, 0.0001)]

In [30]:
MODEL_OUTPUT_DIR = '/run/media/root/Windows/Users/agnes/Downloads/data/msmarco/train_results/doc2vec'

In [34]:
def paramsearch_d2v(sample_combis):

    if len(os.listdir(MODEL_OUTPUT_DIR)) != 0:
        print('error: directory not empty')
    else:

        for i, combi in enumerate(sample_combis):

            model_nr = i
            model_str = f'd2v_{model_nr}'
            this_model = train_doc2vec(tagged_documents[:100], 
                                       os.path.join(MODEL_OUTPUT_DIR, model_str),
                                       epochs=combi[0], vector_size=combi[1], window_size=combi[2],
                                       dm=combi[3], min_alpha=combi[4])

            model_loss_str = 'loss_' + model_str

            print('Evaluating...')
            qid_to_vec, docid_to_vec, eval_dev_df = get_dev_eval_data(dev_df, this_model)

            new_eval_df = calc_dev_performance(qid_to_vec, docid_to_vec, eval_dev_df, 
                                               loss_column_name=model_loss_str)

            with open(os.path.join(MODEL_OUTPUT_DIR, 'losses.txt'), 'a') as fo:
                fo.write('{:s}\t{:2.6f}\n'.format(model_str, 
                                                  new_eval_df[model_loss_str].mean()))
            with open(os.path.join(MODEL_OUTPUT_DIR, 'settings.txt'), 'a') as fo:
                fo.write('{:s}\t{:}\n'.format(model_str, 
                                              list(zip(['epochs', 'vector_size', 'window_size', 'dm', 'min_alpha'],
                                                       combi))))

            print('combi {:d} of {:d} done'.format(i+1, len(sample_combis)))


In [31]:
paramsearch_d2v(sample_combis)

INFO:gensim.models.doc2vec:collecting all words and their counts
INFO:gensim.models.doc2vec:PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO:gensim.models.doc2vec:collected 1590 word types and 100 unique tags from a corpus of 100 examples and 2854 words
INFO:gensim.models.word2vec:Loading a fresh vocabulary
INFO:gensim.models.word2vec:effective_min_count=1 retains 1590 unique words (100% of original 1590, drops 0)
INFO:gensim.models.word2vec:effective_min_count=1 leaves 2854 word corpus (100% of original 2854, drops 0)
INFO:gensim.models.word2vec:deleting the raw counts dictionary of 1590 items
INFO:gensim.models.word2vec:sample=0.001 downsamples 18 most-common words
INFO:gensim.models.word2vec:downsampling leaves estimated 2813 word corpus (98.6% of prior 2854)
INFO:gensim.models.base_any2vec:estimated required memory for 1590 words and 40 dimensions: 1319800 bytes
INFO:gensim.models.word2vec:resetting layer weights
INFO:gensim.models.base_any2vec:training m

Evaluating...
calc dev performance: processed 1000 of 4500 rows
calc dev performance: processed 2000 of 4500 rows
calc dev performance: processed 3000 of 4500 rows


INFO:gensim.models.doc2vec:collecting all words and their counts
INFO:gensim.models.doc2vec:PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO:gensim.models.doc2vec:collected 1590 word types and 100 unique tags from a corpus of 100 examples and 2854 words
INFO:gensim.models.word2vec:Loading a fresh vocabulary
INFO:gensim.models.word2vec:effective_min_count=1 retains 1590 unique words (100% of original 1590, drops 0)
INFO:gensim.models.word2vec:effective_min_count=1 leaves 2854 word corpus (100% of original 2854, drops 0)
INFO:gensim.models.word2vec:deleting the raw counts dictionary of 1590 items
INFO:gensim.models.word2vec:sample=0.001 downsamples 18 most-common words
INFO:gensim.models.word2vec:downsampling leaves estimated 2813 word corpus (98.6% of prior 2854)
INFO:gensim.models.base_any2vec:estimated required memory for 1590 words and 2048 dimensions: 27664760 bytes
INFO:gensim.models.word2vec:resetting layer weights


calc dev performance: processed 4000 of 4500 rows
combi 1 of 20 done


INFO:gensim.models.base_any2vec:training model with 3 workers on 1590 vocabulary and 2048 features, using sg=0 hs=0 sample=0.001 negative=5 window=2
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 1 : training on 2854 raw words (2922 effective words) took 0.0s, 64195 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 2 : training on 2854 raw words (2916 effective words) took 0.0s, 77180 effective words/s
INFO:gensim.models.base_any2vec:worker thread finishe

...

INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 21 : training on 2854 raw words (2911 effective words) took 0.0s, 252869 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 22 : training on 2854 raw words (2912 effective words) took 0.0s, 236755 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models

INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 41 : training on 2854 raw words (2917 effective words) took 0.0s, 282036 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 42 : training on 2854 raw words (2909 effective words) took 0.0s, 278945 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 43 : training on 2854 raw words (2913 effective words) took 0.0s, 19

INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 62 : training on 2854 raw words (2916 effective words) took 0.0s, 266386 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 63 : training on 2854 raw words (2913 effective words) took 0.0s, 239098 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models

INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 82 : training on 2854 raw words (2918 effective words) took 0.0s, 241435 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 83 : training on 2854 raw words (2909 effective words) took 0.0s, 305864 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 84 : training on 2854 raw words (2918 effective words) took 0.0s, 18

Evaluating...
calc dev performance: processed 1000 of 4500 rows
calc dev performance: processed 2000 of 4500 rows
calc dev performance: processed 3000 of 4500 rows
calc dev performance: processed 4000 of 4500 rows
combi 20 of 20 done


In [32]:
# winner is: [('epochs', 100), ('vector_size', 1024), ('window_size', 3), ('dm', 0), ('min_alpha', 0.0001)]

In [39]:
# another round

MODEL_OUTPUT_DIR = '/run/media/root/Windows/Users/agnes/Downloads/data/msmarco/train_results/doc2vec2'

my_epochs = [100, 200, 300, 500]
my_vector_sizes = [1024, 2048]
my_window_sizes = [3, 4]
my_dms = [0]
my_min_alphas = [.0001, .00001]

combis = list(itertools.product(my_epochs, my_vector_sizes, my_window_sizes, my_dms, my_min_alphas))

sample_combis = random.sample(combis, min(20, len(combis)))
print(sample_combis)

paramsearch_d2v(sample_combis)

INFO:gensim.models.doc2vec:collecting all words and their counts
INFO:gensim.models.doc2vec:PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
INFO:gensim.models.doc2vec:collected 1590 word types and 100 unique tags from a corpus of 100 examples and 2854 words
INFO:gensim.models.word2vec:Loading a fresh vocabulary
INFO:gensim.models.word2vec:effective_min_count=1 retains 1590 unique words (100% of original 1590, drops 0)
INFO:gensim.models.word2vec:effective_min_count=1 leaves 2854 word corpus (100% of original 2854, drops 0)
INFO:gensim.models.word2vec:deleting the raw counts dictionary of 1590 items
INFO:gensim.models.word2vec:sample=0.001 downsamples 18 most-common words
INFO:gensim.models.word2vec:downsampling leaves estimated 2813 word corpus (98.6% of prior 2854)
INFO:gensim.models.base_any2vec:estimated required memory for 1590 words and 2048 dimensions: 27664760 bytes
INFO:gensim.models.word2vec:resetting layer weights


[(200, 2048, 3, 0, 1e-05), (500, 2048, 3, 0, 0.0001), (200, 1024, 3, 0, 0.0001), (300, 1024, 4, 0, 1e-05), (200, 1024, 4, 0, 1e-05), (100, 2048, 4, 0, 0.0001), (100, 2048, 4, 0, 1e-05), (500, 2048, 4, 0, 0.0001), (500, 1024, 3, 0, 0.0001), (500, 1024, 4, 0, 0.0001), (200, 2048, 3, 0, 0.0001), (500, 2048, 4, 0, 1e-05), (300, 2048, 4, 0, 1e-05), (100, 1024, 4, 0, 0.0001), (300, 1024, 3, 0, 0.0001), (300, 1024, 4, 0, 0.0001), (200, 1024, 4, 0, 0.0001), (200, 1024, 3, 0, 1e-05), (300, 2048, 3, 0, 0.0001), (500, 1024, 3, 0, 1e-05)]


INFO:gensim.models.base_any2vec:training model with 3 workers on 1590 vocabulary and 2048 features, using sg=1 hs=0 sample=0.001 negative=5 window=3
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 1 : training on 2854 raw words (2921 effective words) took 0.0s, 97887 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 2 : training on 2854 raw words (2910 effective words) took 0.0s, 107777 effective words/s
INFO:gensim.models.base_any2vec:worker thread finish

INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 21 : training on 2854 raw words (2909 effective words) took 0.0s, 85507 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 22 : training on 2854 raw words (2914 effective words) took 0.0s, 108969 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.

INFO:gensim.models.base_any2vec:EPOCH - 41 : training on 2854 raw words (2912 effective words) took 0.0s, 119326 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 42 : training on 2854 raw words (2907 effective words) took 0.0s, 117949 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 43 : training on 2854 raw words (2909 effective words) took 0.0s, 103397 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting fi

...

INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 431 : training on 2854 raw words (2917 effective words) took 0.0s, 252018 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 432 : training on 2854 raw words (2908 effective words) took 0.0s, 214652 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.mode

INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 451 : training on 2854 raw words (2921 effective words) took 0.0s, 244695 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 452 : training on 2854 raw words (2916 effective words) took 0.0s, 242603 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 453 : training on 2854 raw words (2920 effective words) took 0.0s,

INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 472 : training on 2854 raw words (2901 effective words) took 0.0s, 182091 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 473 : training on 2854 raw words (2919 effective words) took 0.0s, 224330 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.mode

INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 492 : training on 2854 raw words (2923 effective words) took 0.0s, 249424 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 493 : training on 2854 raw words (2904 effective words) took 0.0s, 208296 effective words/s
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 2 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 1 more threads
INFO:gensim.models.base_any2vec:worker thread finished; awaiting finish of 0 more threads
INFO:gensim.models.base_any2vec:EPOCH - 494 : training on 2854 raw words (2919 effective words) took 0.0s,

Evaluating...
calc dev performance: processed 1000 of 4500 rows
calc dev performance: processed 2000 of 4500 rows
calc dev performance: processed 3000 of 4500 rows
calc dev performance: processed 4000 of 4500 rows
combi 20 of 20 done


In [ ]:
# winner is: d2v_17	[('epochs', 200), ('vector_size', 1024), ('window_size', 3), ('dm', 0), ('min_alpha', 1e-05)]